# A. Downloading Seasonal Weather Forecasts

The atmosphere can be viewed as a chaotic system in which the future state depends sensitively on the initial conditions, i.e. a slight change in the initial conditions can lead to a significant change in the forecast outputs. The fact that estimates of the current state are inaccurate and that numerical models have inadequacies, leads to forecast errors and uncertainty that grow with increasing forecast lead time. Ensemble forecasting aims at capturing this forecast uncertainty by generating an ensemble of several possible scenarios with the same probability of occurrence. ([Learn more about ensemble prediction](https://www.youtube.com/watch?v=NLhRUun2iso))

![uncertainty](util/images/uncertainty.jpg)

In this Notebook we will learn how to connect to Copernicus Climate Data Store (https://cds.climate.copernicus.eu/) and download ensemble weather forecasts and hindcasts.

## 1. Create an account on the Copernicus Climate Data Store
First of all you need to register on the Copernicus Climate Data Store (CDS): https://cds.climate.copernicus.eu

Once you have created an account copy your user ID (UID) and API key. You can find them in your Copernicus CDS User profile.

In the folder containing this Notebook you will find a file called ".cdsapirc". Copy and paste this file in your "home" folder. On Windows it corresponds to "C:/Users/{your username on Windows}/"
Open the copied file with a text editor, you should see this:

> url: https://cds.climate.copernicus.eu/api/v2

> key: UID:APIkey

Now edit this text and replace UID by your own UID number and APIkey by your own API key number (make sure that both numbers are separated by a colon)
You can also find these intructions in this [link](https://cds.climate.copernicus.eu/api-how-to)

## 2. Import libraries
Now, we need to import the necessary libraries and tools (🚨 in order to run the code like in the box below, place the mouse pointer in the cell, then click on “run cell” button above or press shift + enter).

In [7]:
import cdsapi    #Anaconda prompt -> pip install cdsapi
import netCDF4   #Anaconda prompt -> conda install netcdf4
from netCDF4 import num2date, Dataset
import numpy as np
import pandas as pd
from IPython.display import Image

## 3. Seasonal forecasts systems and datasets for 8 originating centres 
(https://confluence.ecmwf.int/display/CKB/C3S+Seasonal+Forecasts, 8/June/2022)

Please check the details of information that you need such as originating centres and their systems, available datasets from the Table below;

#### - Total precipitation, 1st of every month data
![Seasonal forecasts info](util/images/Forecast_products_info.jpg)

    
There are 33 variables that you can download, however, please note that some of the variables are not supplied by certain originating centres. For more information, contact to Copernicus CDS (https://cds.climate.copernicus.eu/cdsapp#!/dataset/seasonal-original-single-levels?tab=form)
    
#### - Variables
![weather_variables](util/images/weather_variables.jpg)

## 4. Paramaters defining the data to download

This code provides continuous seasonal forecasts dataset download. Enter the detail informaition that you want to download considering the Table and variable information above.

### 4.1. Select the originating centre and the weather variables to download

In [8]:
# Originating centre of the ensemble weather forecast
originating_centre = 'ecmwf'             # ecmwf, ukmo, meteo_france, dwd, cmcc, ncep, jma, eccc
system             = 5                   # See the table above (e.g. ecmwf systems are 4 and 5)
# Weather variables to download
variables = ['2m_temperature', 'total_precipitation', 'evaporation']  # See the variable information above

### 4.2. Forecast dates to download

In [9]:
# Initial dates of the forecast data files that will be downloaded
start_year = 1993   # Download start year
end_year   = 1994   # Download end year
years      = range(start_year, end_year + 1) 

start_month = 1     # Download start month
end_month   = 2     # Download end month
months      = range(start_month, end_month + 1) 

days = 1 # Download start day of the month (i.e. when the forecast data is published, usually the 1st day of 
# the month but this depends on the originating centre)

# From the initial date defined above, we download forecast data with a 7-month lead time. We apply a 24h timestep. 
lead_time       = 5160 # hours, 5160 hours = 7 months approximately
time_step       = 24 # hours
lead_time_hours = range(time_step,lead_time+time_step,time_step)

### 4.3. Spatial extent and catchment name

To define the spatial extent of the data to download, please enter the longitude and latitude in degrees of the squared area that contains the region or catchment of study in the following cell. Also give a name to the catchment of study so it can be later included in the name of the downloaded NetCDF file.

In [10]:
north_lat = 40  # North Latitude (in degrees)
south_lat = 33  # South Latitude (in degrees)
west_lon  = 126 # West Longitude (in degrees)
east_lon  = 129 # East Longitude (in degrees)
catch_name = 'A'  

## 5. Download dataset

Now that we have defined all the necessary parameters we need to run the code below to download all the data. In each loop iteration the code will download 7-month daily data for each defined initial date. 🚨**Please note that this can take several minutes or hours** as each request will be queued. If you download a large dataset, we recommend to run this Notebook at night or when the PC is not in use.

Connection to Copernicus CDS may be lost so please monitor the progress of the download. If connection is lost, please try again updating the start year or month to avoid downloading data that have been already downloaded.

In [11]:
for year in years:
    for month in months:
        c = cdsapi.Client()
        file_location = 'assessment/' + str(originating_centre.upper()) + '/' # folder location where to download the files
        file_name = catch_name + '_' + str(year) + '_' + str(month).zfill(2)+ '_' + str(originating_centre) + '.nc'
        
        # Here we define the parameters to download the forecast
        c.retrieve(
            'seasonal-original-single-levels',
            {
            'format':             'netcdf', # file format
            'originating_centre': originating_centre,
            'system':             system,
            'variable':           variables, # weather variables to download
            'year':               year,
            'month':              month,
            'day':                days,
            'leadtime_hour':      list(lead_time_hours),
            'area':               [north_lat, west_lon, south_lat, east_lon], # If you wish to download global data remove this line
            },
        file_location+file_name) # here we specify the location and name of the file to be saved

2024-07-12 14:51:08,728 INFO Welcome to the CDS
2024-07-12 14:51:08,728 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-original-single-levels
2024-07-12 14:51:08,989 INFO Request is completed
2024-07-12 14:51:08,989 INFO Downloading https://download-0006-clone.copernicus-climate.eu/cache-compute-0006/cache/data6/adaptor.mars.external-1720784773.8429801-3062-17-25636b00-dd4e-4cb2-b1d9-de10ac5889fe.nc to assessment/ECMWF/A_1993_01_ecmwf.nc (1010.5K)
2024-07-12 14:51:09,450 INFO Download rate 2.1M/s                                                                      
2024-07-12 14:51:09,540 INFO Welcome to the CDS
2024-07-12 14:51:09,540 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-original-single-levels
2024-07-12 14:51:09,683 INFO Request is completed
2024-07-12 14:51:09,684 INFO Downloading https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data6/adaptor.mars.external-1720785393.0936604-26017